# Everyday pandas

Pandas is probably the most used python library by any Data Scientist. It is the library that we use it everyday at work and `import pandas as pd` is usually the first thing we type into a notebook. Even though it's such a common tool, I feel like I'm always learning about new functions and ways to explore data in pandas that I didn't know of, specially looking back at how I used it when I first started interning and how I use it now. 

I thought about writing this as a survey on ways to explore data with pandas, with some of the things I learned over the years. Most of it was just things I learned looking at other people's code and with stackoverflow.

In [1]:
import pandas as pd 

#| echo: false


functions

- query 
- qcut
- pivot
- chain functions 
- unstack - migration analysis 
- assign
- style - never run out of it 
- pipe - putting it all together 


In [2]:
#| echo: false
import re 

df = pd.read_csv('fifa/WorldCupMatches.csv').dropna()

new_columns = ['_'.join(x) for x in df.columns.str.split(' ').to_list()]
df.columns =  [x.lower() for x in new_columns]

df = df.assign(year = lambda d: d['year'].astype('int'))
df = df.drop_duplicates()

For this post, I am going to use a [dataset](
https://www.kaggle.com/datasets/abecklas/fifa-world-cup) on matches played on World Cups. Here is a sample of the data:

In [3]:
df.head()

,year,datetime,stage,stadium,city,home_team_name,home_team_goals,away_team_goals,away_team_name,win_conditions,attendance,half-time_home_goals,half-time_away_goals,referee,assistant_1,assistant_2,roundid,matchid,home_team_initials,away_team_initials
0,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4.0,1.0,Mexico,,4444.0,3.0,0.0,LOMBARDI Domingo (URU),CRISTOPHE Henry (BEL),REGO Gilberto (BRA),201.0,1096.0,FRA,MEX
1,1930,13 Jul 1930 - 15:00,Group 4,Parque Central,Montevideo,USA,3.0,0.0,Belgium,,18346.0,2.0,0.0,MACIAS Jose (ARG),MATEUCCI Francisco (URU),WARNKEN Alberto (CHI),201.0,1090.0,USA,BEL
2,1930,14 Jul 1930 - 12:45,Group 2,Parque Central,Montevideo,Yugoslavia,2.0,1.0,Brazil,,24059.0,2.0,0.0,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA),201.0,1093.0,YUG,BRA
3,1930,14 Jul 1930 - 14:50,Group 3,Pocitos,Montevideo,Romania,3.0,1.0,Peru,,2549.0,1.0,0.0,WARNKEN Alberto (CHI),LANGENUS Jean (BEL),MATEUCCI Francisco (URU),201.0,1098.0,ROU,PER
4,1930,15 Jul 1930 - 16:00,Group 1,Parque Central,Montevideo,Argentina,1.0,0.0,France,,23409.0,0.0,0.0,REGO Gilberto (BRA),SAUCEDO Ulises (BOL),RADULESCU Constantin (ROU),201.0,1085.0,ARG,FRA


## `query`

`query` is really just a way to specify how you want to filter the dataset. If you want to select the games played by Brazil as a home team and that Brazil scored 5 goals or more, you can achieve this with the following code:

In [4]:
# Select only BRA plays as home team and when it scored 5 goals or more 
df.query('home_team_initials == "BRA" & home_team_goals >= 5')

,year,datetime,stage,stadium,city,home_team_name,home_team_goals,away_team_goals,away_team_name,win_conditions,attendance,half-time_home_goals,half-time_away_goals,referee,assistant_1,assistant_2,roundid,matchid,home_team_initials,away_team_initials
40,1938,05 Jun 1938 - 17:30,First round,Stade de la Meinau,Strasbourg,Brazil,6.0,5.0,Poland,Brazil win after extra time,13452.0,0.0,0.0,EKLIND Ivan (SWE),POISSANT Louis (FRA),KISSENBERGER Ernest (FRA),206.0,1150.0,BRA,POL
70,1950,09 Jul 1950 - 15:00,Group 6,Maracan� - Est�dio Jornalista M�rio Filho,Rio De Janeiro,Brazil,7.0,1.0,Sweden,,138886.0,3.0,0.0,ELLIS Arthur (ENG),GARCIA Prudencio (USA),DE LA SALLE Charles (FRA),209.0,1189.0,BRA,SWE
71,1950,13 Jul 1950 - 15:00,Group 6,Maracan� - Est�dio Jornalista M�rio Filho,Rio De Janeiro,Brazil,6.0,1.0,Spain,,152772.0,3.0,0.0,LEAFE Reginald (ENG),MITCHELL George (SCO),DA COSTA VIEIRA Jose (POR),209.0,1186.0,BRA,ESP
77,1954,16 Jun 1954 - 18:00,Group 1,Charmilles,Geneva,Brazil,5.0,0.0,Mexico,,13470.0,4.0,0.0,WYSSLING Paul (SUI),SCHONHOLZER Ernest (SUI),DA COSTA VIEIRA Jose (POR),211.0,1249.0,BRA,MEX
133,1958,24 Jun 1958 - 19:00,Semi-finals,Rasunda Stadium,Solna,Brazil,5.0,2.0,France,,27100.0,2.0,1.0,GRIFFITHS Benjamin (WAL),WYSSLING Paul (SUI),LEAFE Reginald (ENG),488.0,1340.0,BRA,FRA
135,1958,29 Jun 1958 - 15:00,Final,Rasunda Stadium,Solna,Brazil,5.0,2.0,Sweden,,49737.0,2.0,1.0,GUIGUE Maurice (FRA),DUSCH Albert (GER),GARDEAZABAL Juan (ESP),3482.0,1343.0,BRA,SWE


It is very similar to what we would using `loc`, and they are essencially the same thing:

In [5]:
df.loc[(df.home_team_initials == "BRA") & (df.home_team_goals >= 5)]

,year,datetime,stage,stadium,city,home_team_name,home_team_goals,away_team_goals,away_team_name,win_conditions,attendance,half-time_home_goals,half-time_away_goals,referee,assistant_1,assistant_2,roundid,matchid,home_team_initials,away_team_initials
40,1938,05 Jun 1938 - 17:30,First round,Stade de la Meinau,Strasbourg,Brazil,6.0,5.0,Poland,Brazil win after extra time,13452.0,0.0,0.0,EKLIND Ivan (SWE),POISSANT Louis (FRA),KISSENBERGER Ernest (FRA),206.0,1150.0,BRA,POL
70,1950,09 Jul 1950 - 15:00,Group 6,Maracan� - Est�dio Jornalista M�rio Filho,Rio De Janeiro,Brazil,7.0,1.0,Sweden,,138886.0,3.0,0.0,ELLIS Arthur (ENG),GARCIA Prudencio (USA),DE LA SALLE Charles (FRA),209.0,1189.0,BRA,SWE
71,1950,13 Jul 1950 - 15:00,Group 6,Maracan� - Est�dio Jornalista M�rio Filho,Rio De Janeiro,Brazil,6.0,1.0,Spain,,152772.0,3.0,0.0,LEAFE Reginald (ENG),MITCHELL George (SCO),DA COSTA VIEIRA Jose (POR),209.0,1186.0,BRA,ESP
77,1954,16 Jun 1954 - 18:00,Group 1,Charmilles,Geneva,Brazil,5.0,0.0,Mexico,,13470.0,4.0,0.0,WYSSLING Paul (SUI),SCHONHOLZER Ernest (SUI),DA COSTA VIEIRA Jose (POR),211.0,1249.0,BRA,MEX
133,1958,24 Jun 1958 - 19:00,Semi-finals,Rasunda Stadium,Solna,Brazil,5.0,2.0,France,,27100.0,2.0,1.0,GRIFFITHS Benjamin (WAL),WYSSLING Paul (SUI),LEAFE Reginald (ENG),488.0,1340.0,BRA,FRA
135,1958,29 Jun 1958 - 15:00,Final,Rasunda Stadium,Solna,Brazil,5.0,2.0,Sweden,,49737.0,2.0,1.0,GUIGUE Maurice (FRA),DUSCH Albert (GER),GARDEAZABAL Juan (ESP),3482.0,1343.0,BRA,SWE


I feel that mostly the advantages of using `query` are related to readability and general code organization. I just think is a nicer, and often, a faster way to write your code. 

If we want to reference variables in the environment, we can achieve that with `query` using `@` before it, such as in: 

In [6]:
min_goals = 5
df.query('home_team_initials == "BRA" & home_team_goals >= @min_goals')

,year,datetime,stage,stadium,city,home_team_name,home_team_goals,away_team_goals,away_team_name,win_conditions,attendance,half-time_home_goals,half-time_away_goals,referee,assistant_1,assistant_2,roundid,matchid,home_team_initials,away_team_initials
40,1938,05 Jun 1938 - 17:30,First round,Stade de la Meinau,Strasbourg,Brazil,6.0,5.0,Poland,Brazil win after extra time,13452.0,0.0,0.0,EKLIND Ivan (SWE),POISSANT Louis (FRA),KISSENBERGER Ernest (FRA),206.0,1150.0,BRA,POL
70,1950,09 Jul 1950 - 15:00,Group 6,Maracan� - Est�dio Jornalista M�rio Filho,Rio De Janeiro,Brazil,7.0,1.0,Sweden,,138886.0,3.0,0.0,ELLIS Arthur (ENG),GARCIA Prudencio (USA),DE LA SALLE Charles (FRA),209.0,1189.0,BRA,SWE
71,1950,13 Jul 1950 - 15:00,Group 6,Maracan� - Est�dio Jornalista M�rio Filho,Rio De Janeiro,Brazil,6.0,1.0,Spain,,152772.0,3.0,0.0,LEAFE Reginald (ENG),MITCHELL George (SCO),DA COSTA VIEIRA Jose (POR),209.0,1186.0,BRA,ESP
77,1954,16 Jun 1954 - 18:00,Group 1,Charmilles,Geneva,Brazil,5.0,0.0,Mexico,,13470.0,4.0,0.0,WYSSLING Paul (SUI),SCHONHOLZER Ernest (SUI),DA COSTA VIEIRA Jose (POR),211.0,1249.0,BRA,MEX
133,1958,24 Jun 1958 - 19:00,Semi-finals,Rasunda Stadium,Solna,Brazil,5.0,2.0,France,,27100.0,2.0,1.0,GRIFFITHS Benjamin (WAL),WYSSLING Paul (SUI),LEAFE Reginald (ENG),488.0,1340.0,BRA,FRA
135,1958,29 Jun 1958 - 15:00,Final,Rasunda Stadium,Solna,Brazil,5.0,2.0,Sweden,,49737.0,2.0,1.0,GUIGUE Maurice (FRA),DUSCH Albert (GER),GARDEAZABAL Juan (ESP),3482.0,1343.0,BRA,SWE


With `query`, we can also use the `in` operator to select only the values within a list: 

In [7]:
teams = ['BRA', 'FRA']
df.query('home_team_initials in @teams & home_team_goals >= @min_goals')

,year,datetime,stage,stadium,city,home_team_name,home_team_goals,away_team_goals,away_team_name,win_conditions,attendance,half-time_home_goals,half-time_away_goals,referee,assistant_1,assistant_2,roundid,matchid,home_team_initials,away_team_initials
40,1938,05 Jun 1938 - 17:30,First round,Stade de la Meinau,Strasbourg,Brazil,6.0,5.0,Poland,Brazil win after extra time,13452.0,0.0,0.0,EKLIND Ivan (SWE),POISSANT Louis (FRA),KISSENBERGER Ernest (FRA),206.0,1150.0,BRA,POL
70,1950,09 Jul 1950 - 15:00,Group 6,Maracan� - Est�dio Jornalista M�rio Filho,Rio De Janeiro,Brazil,7.0,1.0,Sweden,,138886.0,3.0,0.0,ELLIS Arthur (ENG),GARCIA Prudencio (USA),DE LA SALLE Charles (FRA),209.0,1189.0,BRA,SWE
71,1950,13 Jul 1950 - 15:00,Group 6,Maracan� - Est�dio Jornalista M�rio Filho,Rio De Janeiro,Brazil,6.0,1.0,Spain,,152772.0,3.0,0.0,LEAFE Reginald (ENG),MITCHELL George (SCO),DA COSTA VIEIRA Jose (POR),209.0,1186.0,BRA,ESP
77,1954,16 Jun 1954 - 18:00,Group 1,Charmilles,Geneva,Brazil,5.0,0.0,Mexico,,13470.0,4.0,0.0,WYSSLING Paul (SUI),SCHONHOLZER Ernest (SUI),DA COSTA VIEIRA Jose (POR),211.0,1249.0,BRA,MEX
105,1958,08 Jun 1958 - 19:00,Group 2,Idrottsparken,Norrk�Ping,France,7.0,3.0,Paraguay,,16518.0,2.0,2.0,GARDEAZABAL Juan (ESP),GRIFFITHS Benjamin (WAL),BROZZI Juan (ARG),220.0,1386.0,FRA,PAR
133,1958,24 Jun 1958 - 19:00,Semi-finals,Rasunda Stadium,Solna,Brazil,5.0,2.0,France,,27100.0,2.0,1.0,GRIFFITHS Benjamin (WAL),WYSSLING Paul (SUI),LEAFE Reginald (ENG),488.0,1340.0,BRA,FRA
134,1958,28 Jun 1958 - 17:00,Match for third place,Nya Ullevi,Gothenburg,France,6.0,3.0,Germany FR,,32483.0,3.0,1.0,BROZZI Juan (ARG),ELLIS Arthur (ENG),LUNDELL Bengt (SWE),3483.0,1382.0,FRA,FRG
135,1958,29 Jun 1958 - 15:00,Final,Rasunda Stadium,Solna,Brazil,5.0,2.0,Sweden,,49737.0,2.0,1.0,GUIGUE Maurice (FRA),DUSCH Albert (GER),GARDEAZABAL Juan (ESP),3482.0,1343.0,BRA,SWE


## `assign`

Here is what we can find about the `assign` function on the pandas documentation: 

```Returns a new object with all original columns in addition to new ones. Existing columns that are re-assigned will be overwritten.```

So `assign` is literally just a different way to *assign* new columns to our dataframe, but really the great thing is that we can create multiple columns at once, in which a new column can also depend on one that was just created. 

Let's use as an example a case where we want to query the matches called a "big goal match". This type of match represents the games in which the total number of goals were bigger than 6. One way to go about that is to create a new column `total_goals` that sums the goals scored by the home team and the away team, and create a new column `big_goals_match` that can be either 0 or 1 to flag if the game was considered a "big goal match".

Here is how we can perform this with `assign`:

In [8]:
goals_matches = df.assign(
                    total_goals     = lambda d: d['home_team_goals'] + d['away_team_goals'], 
                    big_goals_match = lambda d: (d['total_goals'] > 5).astype('int')
                )[['home_team_initials', 'away_team_initials', 'home_team_goals', 'away_team_goals', 'total_goals', 'big_goals_match']]

goals_matches.query('big_goals_match == 1').head()

,home_team_initials,away_team_initials,home_team_goals,away_team_goals,total_goals,big_goals_match
10,ARG,MEX,6.0,3.0,9.0,1
15,ARG,USA,6.0,1.0,7.0,1
16,URU,YUG,6.0,1.0,7.0,1
17,URU,ARG,4.0,2.0,6.0,1
19,HUN,EGY,4.0,2.0,6.0,1


##### What if we wanted to get the top 10 matches that are a "big goal match" with the smallest difference between the home and away team?

Before going straight to the code, one thing I learned in pandas was to organize my code within the parenthesis so that we can escape lines when adding new methods to the existing dataframe. In a code like this, you know very easily what is being computed at each step. It's easier to read, and in case you want to test something new, you can simply comment or uncomment an existing line:

In [9]:
(df.assign( 
    # Add new columns to existing dataframe 
        total_goals =     lambda d: d['home_team_goals'] + d['away_team_goals'], 
        big_goals_match = lambda d: (d['total_goals'] > 5).astype('int'), 
        goal_diff =       lambda d: abs(d['home_team_goals'] - d['away_team_goals'])
    )

    # Select only the big goals match 
    .query('big_goals_match == 1')

    # sort values by goal_diff
    .sort_values('goal_diff', ascending = True)

    # select the first 10 lines
    .head(10)

    # Reset index so it numbers the data from 0 to 9
    .reset_index(drop = True)

    # Pick important columns for display
    [['home_team_initials', 'away_team_initials', 'home_team_goals', 'away_team_goals', 'total_goals', 'big_goals_match', 'goal_diff']]
)

,home_team_initials,away_team_initials,home_team_goals,away_team_goals,total_goals,big_goals_match,goal_diff
0,URS,COL,4.0,4.0,8.0,1,0.0
1,ENG,BEL,4.0,4.0,8.0,1,0.0
2,FRG,FRA,3.0,3.0,6.0,1,0.0
3,PAR,YUG,3.0,3.0,6.0,1,0.0
4,SEN,URU,3.0,3.0,6.0,1,0.0
5,CUB,ROU,3.0,3.0,6.0,1,0.0
6,BRA,POL,6.0,5.0,11.0,1,1.0
7,URS,BEL,3.0,4.0,7.0,1,1.0
8,ITA,FRG,4.0,3.0,7.0,1,1.0
9,URU,ENG,4.0,2.0,6.0,1,2.0


## `cut` and `qcut`

Very often when we run some type of analysis, it's interesting to see how the data is distributed acording to a certain variable. A nice way to do this is to simply add bins to the data, and pandas makes it very easy to do that with `cut` and `qcut`. The difference between them is that `cut` let's you specify the exact bin edges that you want to cut the data, and `qcut` uses the quantiles to do that. I personally use `qcut` way more often, since it already splits the data in equal size bins.

Let's try to answer the following question: **"Are the matches in the top 10% attendance of all time made only by the final stages in the World Cup?"**

To answer this, let's first create the bins with `qcut`. In order to understand what is the top 10% in attendence, we need to split the data in 10 equally sized parts, here is how we can do that:

In [10]:
labels_bins = [f'p{x}' for x in range(0, 100, 10)]

df = (df.assign(
        bins_attendance = lambda d: pd.qcut(d['attendance'], q = 10, labels = labels_bins)
    )
)

# Check the distribution for each bin
df.bins_attendance.value_counts(normalize = True)

p60    0.101796
p0     0.100599
p20    0.100599
p40    0.100599
p90    0.100599
p10    0.099401
p30    0.099401
p50    0.099401
p80    0.099401
p70    0.098204
Name: bins_attendance, dtype: float64

As expected, each bin has around 10% of the data 

Now, answering the question:

In [11]:
highest_attendence = (df.groupby(['stage'])
                        ['bins_attendance']
                        .value_counts()
                        .reset_index(name = 'counts')
                        .query('bins_attendance == "p90" & counts > 0')
                        .sort_values('counts', ascending = False)
                        .reset_index(drop = True)
                    )

highest_attendence

,stage,bins_attendance,counts
0,Group 1,p90,12
1,Group B,p90,11
2,Final,p90,9
3,Semi-finals,p90,8
4,Group A,p90,7
5,Round of 16,p90,7
6,Quarter-finals,p90,6
7,Group 2,p90,4
8,Group E,p90,4
9,Match for third place,p90,4


As we can see, the top 10% in attendence is made by lots of different World Cup stages, including the first ones.

## Pivot table
Pivot table is a *very* common transformation. If you know excel, you're probably already familiar with it and use it a lot as well. This method is very useful when we want to check how two variables (or more) affect another one with a custom statistic. Sounds a bit confusing, so let's illustrate that with this question:

- How many goals each stage had on average in the last 3 world cups?

So what we want would be a dataframe that had columns as the years (indicading the last 3 world cups), the index as the stages, and the values as the mean goals scored in each stage/year. There are different ways to construct this, but `pivot_table` really makes it easy, here's how to do that:

In [12]:
# Add column with the total number of goals scored 
df = df.assign(
    total_goals = lambda d: d['home_team_goals'] + d['away_team_goals']
)

# Build pivot table
df_avg_goals = (pd.pivot_table(
                        df.query('year >= 2006'), 
                        columns = 'year', 
                        index   = 'stage', 
                        values  = 'total_goals', 
                        aggfunc = 'mean'
                    ).round(2)
                )

df_avg_goals

year,2006,2010,2014
stage,,,
Final,2.00,1.00,1.00
Group A,3.00,1.83,3.00
Group B,1.67,2.83,3.67
Group C,3.00,1.50,2.83
Group D,2.00,2.00,2.00
Group E,2.33,2.33,3.17
Group F,2.67,2.17,2.33
Group G,1.83,2.83,3.17
Group H,3.00,1.33,2.50


### `style` - never run out of it! 
It's hard to collect insights looking at the last table quickly. With `style` we can set different ways to color our dataframe. To simply create a color scale, we can use the method `background_gradient`:

In [13]:
(df_avg_goals
    .style
    .format(na_rep='', precision=2)
    .background_gradient()
)

year,2006,2010,2014
stage,,,
Final,2.00,1.00,1.00
Group A,3.00,1.83,3.00
Group B,1.67,2.83,3.67
Group C,3.00,1.50,2.83
Group D,2.00,2.00,2.00
Group E,2.33,2.33,3.17
Group F,2.67,2.17,2.33
Group G,1.83,2.83,3.17
Group H,3.00,1.33,2.50


Much better! 

The Style object from pandas has a lot of different ways to customize a dataframe. The `background_gradient` method already satisfies most of my needs for styling, but if you want to check more options, the [documentation](https://pandas.pydata.org/docs/user_guide/style.html) has a lot of great examples.

## `pipe` - putting it all together! 

I first learned about this method with [calmcode.io](https://calmcode.io/pandas-pipe/introduction.html). There is a series of very nice videos that I definitely recommend checking out! 

With pipe, we can create several of different functions to process our dataset. The idea is that each function should be responsible for one step of the dataset processing. For example, it's very common to fix the data types, then add some new variables to the data, perform some type of scaling, etc. So, instead of doing something like this: 

```{python}
df1 = df.astype(type_schema)

df2 = df1.copy()
df2['total_goals'] = df2['home_team_goals'] + df2['away_team_goals']

df3 = df2.copy()
data_scaled = scaler.fit_transform(df3[feature_list])
```

We can do something like this: 

```{python}
def fix_dtypes(dataf, type_schema): 
    return dataf.astype(type_schema)

def add_total_goals(dataf): 
    return dataf.assign(
        lambda d: d['home_team_goals'] + d['away_team_goals']
    )

def scale_dataset(dataf, features): 
    X = scaler.fit_transform(dataf[features])
    return pd.DataFrame(X, index = dataf.index, columns = features)
```

And then simply call pipe to chain the functions:
```{python}
data_scaled = (df.pipe(fix_dtypes, type_schema = type_schema)
                 .pipe(add_total_goals)
                 .pipe(scale_dataset, features = feature_list)
)
```

This can bring a very nice touch to our code for a few reasons: 

- Readability: It's a lot easier to understand and also organize the code. By creating functions for each step, you will probabily spend a bit more time trying to make it nicer and reproducible, rather than just creating a lot of different dataframes for every tiny change you want to make. 
- Practical: If you're running different tests, you can simply comment/uncomment a line and all the other transformations that you need will still be there. 
- Clean code: If you have common steps for several notebooks, you can add the functions to a python file and then import them to the notebook. The code will be a lot cleaner and the notebook will have only the analysis/plots/modeling/etc. In my experience, `pipe` is specially useful when building the preprocessing steps for a Machine Learning model.


Now that we have all that in mind, let's try to answer this question to put everything we learned together:

- On average, a team that won the World Cup scores how many more goals than other teams?

Before anything, let's take a look at our data again:

In [14]:
df.head()

,year,datetime,stage,stadium,city,home_team_name,home_team_goals,away_team_goals,away_team_name,win_conditions,...,half-time_away_goals,referee,assistant_1,assistant_2,roundid,matchid,home_team_initials,away_team_initials,bins_attendance,total_goals
0,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4.0,1.0,Mexico,,...,0.0,LOMBARDI Domingo (URU),CRISTOPHE Henry (BEL),REGO Gilberto (BRA),201.0,1096.0,FRA,MEX,p0,5.0
1,1930,13 Jul 1930 - 15:00,Group 4,Parque Central,Montevideo,USA,3.0,0.0,Belgium,,...,0.0,MACIAS Jose (ARG),MATEUCCI Francisco (URU),WARNKEN Alberto (CHI),201.0,1090.0,USA,BEL,p10,3.0
2,1930,14 Jul 1930 - 12:45,Group 2,Parque Central,Montevideo,Yugoslavia,2.0,1.0,Brazil,,...,0.0,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA),201.0,1093.0,YUG,BRA,p10,3.0
3,1930,14 Jul 1930 - 14:50,Group 3,Pocitos,Montevideo,Romania,3.0,1.0,Peru,,...,0.0,WARNKEN Alberto (CHI),LANGENUS Jean (BEL),MATEUCCI Francisco (URU),201.0,1098.0,ROU,PER,p0,4.0
4,1930,15 Jul 1930 - 16:00,Group 1,Parque Central,Montevideo,Argentina,1.0,0.0,France,,...,0.0,REGO Gilberto (BRA),SAUCEDO Ulises (BOL),RADULESCU Constantin (ROU),201.0,1085.0,ARG,FRA,p10,1.0


One way to go about this, is to create a dataframe that contains how many goals each team scored in the world cup, add a column `is_world_cup_winner` and then compare the winners with the others. 

To identify the winners, we can check who won the final match by comparing the number of goals, but the problem with the dataset is that when the final was decided on the penalties, the winner will be on the description `win_conditions`:

In [15]:
df.query('year == 2006 & stage == "Final"')[['home_team_name',
                                             'home_team_goals',
                                             'away_team_goals', 
                                             'away_team_name', 
                                             'win_conditions'
                                             ]]

,home_team_name,home_team_goals,away_team_goals,away_team_name,win_conditions
707,Italy,1.0,1.0,France,Italy win on penalties (5 - 3)


So, we will also have to correct for that. 

Here is the pipeline we will build: 
- `start_pipe`: just start by copying the dataframe since the `pipe` method modifies the original df 
- `total_goals_by_year`: create a df with how many goals each country scored each year
- `world_cup_winners`: query the countries that were in the "Final" stage in each year and select the country with more goals. Then, apply `correct_winners` to correct the winner country when the World Cup was decided on the penalties:

In [16]:
def start_pipe(dataf): 
    return dataf.copy()
    
def total_goals_by_year(dataf):
    away_team_goals = (dataf.groupby(['year', 'away_team_name'])
                        ['away_team_goals']
                        .sum()
                        .to_frame()
                    )
    home_team_goals = (dataf.groupby(['year', 'home_team_name'])
                        ['home_team_goals']
                        .sum()
                        .to_frame()
                    )
    
    return (pd.concat([away_team_goals,
                       home_team_goals], axis = 1
                ).fillna(0)
                .assign(total_goals = lambda d: d['away_team_goals'] + d['home_team_goals'])
            )[['total_goals']]

def world_cup_winners(dataf): 
   return (dataf.query('stage == "Final"')
                .assign(winner_team = lambda d: d.apply(
                                    lambda x: x['home_team_name'] if x['home_team_goals'] > x['away_team_goals'] else x['away_team_name'], axis = 1
                                )
                        )
        )

def correct_winners(description): 
    match = re.search(r'\b(\w+)\s+win\b', description)
    if match != None: 
        return match.group(1)
    else: 
        return ''
    
def wc_winners_corrected(dataf): 
    return (dataf.assign(teams_corrected = lambda d: d.win_conditions.apply(correct_winners), 
                         winner_team_corrected = lambda d: d.apply(
                            lambda x: x['winner_team'] if x['teams_corrected'] == '' else x['teams_corrected'], axis = 1
                        )

                    ).reset_index(drop = True)
    )[['year', 'winner_team', 'teams_corrected', 'winner_team_corrected']]

In [17]:
winners = (df.pipe(start_pipe)
            .pipe(world_cup_winners)
            .pipe(wc_winners_corrected)
         )

winners.tail()

,year,winner_team,teams_corrected,winner_team_corrected
14,1998,France,,France
15,2002,Brazil,,Brazil
16,2006,France,Italy,Italy
17,2010,Spain,Spain,Spain
18,2014,Germany,Germany,Germany


In [18]:
country_goals_by_year = (df.pipe(start_pipe)
                           .pipe(total_goals_by_year))

country_goals_by_year.tail()

total_goals
year                     
1998 Brazil          14.0
     Germany          8.0
     Italy            8.0
     Romania          4.0
     Spain            8.0

Finally, we can merge the datasets so we have how many goals were scored by each winner team:

In [19]:
winners_goals = (   # merge winner's information with total goals by each country on that year
                    pd.merge(
                        winners[['year', 'winner_team_corrected']],
                        (country_goals_by_year.reset_index()
                            .rename(columns = {'level_1':'team_name'})
                        ),                     
                        on = 'year'
                    )
                    # assign a flag to world cup winners based on winners df 
                    .assign(is_world_cup_winner = lambda d: (d['team_name'] == d['winner_team_corrected']).astype('int'))

                    # keep relevant columns
                    [['year', 'team_name', 'total_goals', 'is_world_cup_winner']]
            )
winners_goals.head()

,year,team_name,total_goals,is_world_cup_winner
0,1930,Argentina,18.0,0
1,1930,Belgium,0.0,0
2,1930,Bolivia,0.0,0
3,1930,Brazil,5.0,0
4,1930,Chile,5.0,0


Now, we can simply calculate the mean total goals by a team when they have won the World Cup: 

In [20]:
avg_goals = (winners_goals
                .groupby('is_world_cup_winner')
                [['total_goals']]
                .mean().T
            )

avg_goals

is_world_cup_winner,0,1
total_goals,5.103797,14.315789


In [21]:
avg_diff_goals = (avg_goals[1] - avg_goals[0]).iloc[0].round(2)

print(f'On average, a team that won the World Cup scores {avg_diff_goals} more goals than other teams')

On average, a team that won the World Cup scores 9.21 more goals than other teams


## Final thoughts 

Pandas is for sure one of the most important libraries I work with everyday, and still I keep finding new ways to use it. There are a dozen other topics we didn't cover here, but I just wanted to do a general overview on some of the very important things I learned with the library and that really help me on a daily basis. 

If you are looking for more resources, a lot of the ideas here come from the very good [Modern Pandas](https://tomaugspurger.net/posts/modern-1-intro/) writing series by Tom Augspurger. For portuguese speakers, I also have a [post on my Medium page](https://medium.com/turing-talks/manipula%C3%A7%C3%A3o-de-s%C3%A9ries-temporais-com-pandas-db4ee39a0c1b) about pandas for Time Series analysis.